In [2]:
import pandas as pd
import numpy as np

In [3]:
# 데이터 불러오기
trade_df = pd.read_csv('../trades.csv', index_col='idx')        # 실거래가 파일
complex_df = pd.read_csv('../complexes.csv', index_col='idx')   # 단지 정보 파일
pyeong_df = pd.read_csv('../pyeongs.csv', index_col='idx')      # 평 정보 파일

In [ ]:
trade_df.shape

In [4]:
# pyeong 데이터에서 complexNo와  exclusiveArea가 동일한 중복 행들을 모두 제거
pyeong_df.drop_duplicates(subset=['complexNo', 'exclusiveArea'], inplace=True)

In [5]:
# pyeong과 complex 조인
# complexNo가 같은 행끼리 조인
pyeong_complex_df = pd.merge(
                        pyeong_df, 
                        complex_df, 
                        left_on='complexNo',
                        right_on='complexNo',
                        how='inner'
                    )

In [6]:
# completionYear 컬럼이 NaN인 행을 모두 제거
pyeong_complex_df.dropna(subset = ['completionYear'], inplace=True)
print(pyeong_complex_df.head(3))

       Unnamed: 0_x     id  complexNo  pyeongNo  supplyArea pyeongName  \
0                 0      1          1         1       82.29         82   
1             12174  12175          1         2       89.83         89   
2             23435  23436          1         4      103.32        103   
3             32898  32899          1         3      104.86        104   
4                 1      2         10         1       55.51         55   
...             ...    ...        ...       ...         ...        ...   
44406         12173  12174       9999         1       44.91         44   
44407         23434  23435       9999         2       48.38         48   
44408         32899  32900       9999         4       68.92        68A   
44409         40280  40281       9999         5       70.40        70B   
44410         45896  45897       9999         3       72.20         72   

       supplyPyeong pyeongName2  exclusiveArea  exclusivePyeong  ...  \
0             24.89          24        

In [8]:
# 단지-평 정보 데이터에서 detailAddress, exclusiveArea, completionYear 컬럼이 모두 일치하는 중복되는 행 삭제
pyeong_complex_df.drop_duplicates(subset=['detailAddress', 'exclusiveArea', 'completionYear'], inplace=True)

In [ ]:
# 단지-평 정보 데이터 길이
print(pyeong_complex_df.shape)

In [9]:
# '실거래가 데이터'와 '단지데이터+평데이터' 조인
# 주소, 면적, 건축년도가 모두 같은 행끼리 결합
pyeong_complex_df['fk'] = pyeong_complex_df['detailAddress'] + ' ' \
                        + pyeong_complex_df['exclusiveArea'].astype('str') + ' ' \
                        + pyeong_complex_df['completionYear'].astype('int64').astype('str')

trade_df['fk'] = trade_df['detailAddress'] + ' ' \
                + trade_df['exclusiveArea'].astype('str') + ' ' \
                + trade_df['constructYear'].astype('str')

raw_train_df = pd.merge(trade_df, 
                        pyeong_complex_df, 
                        left_on='fk', 
                        right_on='fk',
                        how='left'
                       )

print(raw_train_df)

         Unnamed: 0      address_x detailAddress_x  detailAddressMain  \
0                 0  서울특별시 도봉구 방학동             713                713   
1                 1  서울특별시 강남구 개포동           655-2                655   
2                 2  서울특별시 강남구 개포동           655-2                655   
3                 3  서울특별시 강남구 개포동           658-1                658   
4                 4  서울특별시 강남구 개포동           658-1                658   
...             ...            ...             ...                ...   
1155880       74921  서울특별시 중랑구 중화동             450                450   
1155881       74922  서울특별시 중랑구 중화동          274-51                274   
1155882       74923  서울특별시 중랑구 중화동          274-75                274   
1155883       74924  서울특별시 중랑구 중화동              11                 11   
1155884       74925  서울특별시 중랑구 중화동            11-7                 11   

         detailAddressSub   complexName_x  exclusiveArea_x  \
0                       0      브라운스톤방학아파트            84.90   

In [10]:
# 불필요한 컬럼 제거
raw_train_df = raw_train_df[['detailAddress_x', 'complexName_x', 'exclusiveArea_x', 'contactDate', 'price', 'floor', 'constructYear',
       'roadAddress_x', 'complexNo', 'pyeongNo', 'supplyArea', 'supplyPyeong', 'exclusivePyeong',
       'householdCountByPyeong', 'realEstateTypeCode_x', 'exclusiveRate', 'entranceType', 'roomCnt', 'bathroomCnt',
       'complexName_y', 'cortarNo',  'roadAddress_y', 'latitude', 'longitude', 'totalHouseholdCount', 
       'totalLeaseHouseholdCount', 'highFloor', 'lowFloor', 'completionMonth', 'totalDongCount',
       'maxSupplyArea', 'minSupplyArea', 'dealCount', 'rentCount', 'batlRatio', 'btlRatio', 'parkingCountByHousehold', 
       'constructionCompanyName', 'heatMethodTypeCode', 'heatFuelTypeCode', 'address_y']]

raw_train_df.columns = ['detailAddress', 'complexName', 'exclusiveArea', 'contactDate', 'price', 'floor', 'constructYear',
       'roadAddress', 'complexNo', 'pyeongNo', 'supplyArea', 'supplyPyeong', 'exclusivePyeong',
       'householdCountByPyeong', 'realEstateTypeCode', 'exclusiveRate', 'entranceType', 'roomCnt', 'bathroomCnt',
       'complexName', 'cortarNo',  'roadAddress', 'latitude', 'longitude', 'totalHouseholdCount', 
       'totalLeaseHouseholdCount', 'highFloor', 'lowFloor', 'completionMonth', 'totalDongCount',
       'maxSupplyArea', 'minSupplyArea', 'dealCount', 'rentCount', 'batlRatio', 'btlRatio', 'parkingCountByHousehold', 
       'constructionCompanyName', 'heatMethodTypeCode', 'heatFuelTypeCode', 'address']

In [ ]:
# 삭제 전 학습용 데이터 길이
print(raw_train_df.shape)

In [11]:
# 조인되지 않는 데이터 삭제
raw_train_df = raw_train_df[raw_train_df.isna()['address']==False]

raw_train_df.index.name = 'id'

print(raw_train_df)

        detailAddress     complexName  exclusiveArea          contactDate  \
id                                                                          
0                 713      브라운스톤방학아파트          84.90  2005-11-01 00:00:00   
1               655-2  개포2차현대아파트(220)          77.75  2012-05-01 00:00:00   
2               655-2  개포2차현대아파트(220)          77.75  2012-07-09 00:00:00   
3               658-1  개포6차우성아파트1동~8동          67.28  2012-12-26 00:00:00   
4               658-1  개포6차우성아파트1동~8동          54.98  2010-01-28 00:00:00   
...               ...             ...            ...                  ...   
1155867           450            한신1차          50.37  2019-12-20 00:00:00   
1155868           450            한신1차          50.37  2019-12-21 00:00:00   
1155869           450            한신1차          84.87  2019-12-30 00:00:00   
1155881        274-51         한영(101)          84.69  2019-09-03 00:00:00   
1155883            11           현대휴앤미          82.17  2019-01-31 00:00:00   

In [ ]:
# 삭제 후 학습용 데이터 길이
print(raw_train_df.shape)

In [ ]:
# ***************************************** 거리데이터 붙이는 내용 추가해야함 *****************************************

In [12]:
# 학습용 데이터 저장
raw_train_df.to_csv('train_data.csv', encoding='utf-8')